In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

from sklearn.tree import _tree
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [9]:
DATA_PATH   = "../data/rps_1694359904.9626915.csv"
RANDOM_SEED = 0
TEST_SIZE   = 0.2
MODEL_OUT   = "../models/model.js"

### Data Loading

In [10]:
df = pd.read_csv(DATA_PATH)
df.head()

,WRIST_X,WRIST_Y,THUMB_CMC_X,THUMB_CMC_Y,THUMB_MCP_X,THUMB_MCP_Y,THUMB_IP_X,THUMB_IP_Y,THUMB_TIP_X,THUMB_TIP_Y,...,RING_FINGER_TIP_Y,PINKY_MCP_X,PINKY_MCP_Y,PINKY_PIP_X,PINKY_PIP_Y,PINKY_DIP_X,PINKY_DIP_Y,PINKY_TIP_X,PINKY_TIP_Y,CLASS
0,0.522535,0.584055,0.625936,0.580929,0.718496,0.526368,0.727261,0.450880,0.702420,0.407202,...,0.451068,0.524569,0.375544,0.566660,0.338999,0.550875,0.400552,0.533669,0.423864,0
1,0.522810,0.581773,0.625333,0.581156,0.717528,0.528407,0.730288,0.454104,0.713798,0.408834,...,0.448014,0.523502,0.376928,0.564864,0.340199,0.549492,0.401110,0.531896,0.420952,0
2,0.523001,0.582668,0.624563,0.580880,0.716532,0.527002,0.724324,0.452666,0.699354,0.410737,...,0.447512,0.523334,0.378884,0.564633,0.339105,0.550132,0.400343,0.532758,0.421779,0
3,0.523001,0.582668,0.624563,0.580880,0.716532,0.527002,0.724324,0.452666,0.699354,0.410737,...,0.447512,0.523334,0.378884,0.564633,0.339105,0.550132,0.400343,0.532758,0.421779,0
4,0.522727,0.578590,0.624897,0.577490,0.716129,0.524396,0.725709,0.449820,0.704026,0.404276,...,0.448760,0.523509,0.373285,0.567279,0.339910,0.549760,0.399989,0.530882,0.419607,0


In [11]:
x = df.drop("CLASS",axis=1)
y = df["CLASS"]

xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=TEST_SIZE,random_state=RANDOM_SEED)

### Model Selection

In [113]:
rfc = RandomForestClassifier(random_state=RANDOM_SEED)
parameters = {
                "n_estimators":[100,250,500,1000],
                "max_depth"   :[8,16,32,None]
             }

cv = GridSearchCV(rfc,parameters,cv=5)
cv.fit(xtrain,ytrain)

print(cv.best_params_)

{'max_depth': 16, 'n_estimators': 100}


### Model Training

In [12]:
model = RandomForestClassifier(random_state=RANDOM_SEED,n_estimators=100,max_depth=16).fit(xtrain, ytrain)

In [14]:
y_pred      = model.predict(xtest)
conf_matrix = confusion_matrix(ytest,y_pred,labels=[0,1,2])
conf_matrix

array([[237,   0,   0],
       [  0, 186,   1],
       [  0,   1, 175]])

### Model Exporting to JS

In [24]:
def write(string,append=True,end="\n"):
    string = str(string) + end
    if append:
        with open(MODEL_OUT,'a') as file:
            file.write(string)
    else:
        with open(MODEL_OUT,'a') as file:
            file.write("")
        with open(MODEL_OUT,'w') as file:
            file.write(string)
            


def decode_dt_tojs(dt:'sklearn.tree.DecisionTreeClassifier',in_features:list[str],func_name:str):
    write(f"let {func_name} = ({','.join(in_features)}) =>",end="")
    write("{")
    
    tree_ = dt.tree_
    in_features = [
        in_features[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]
    
    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = in_features[node]
            threshold = tree_.threshold[node]
            write ("{}if ({} <= {})".format(indent, name, threshold)+"{")
            recurse(tree_.children_left[node], depth + 1)
            write("{}".format(indent)+"}")
            write ("{}else".format(indent)+"{")
            recurse(tree_.children_right[node], depth + 1)
            write("{}".format(indent)+"}")
        else:
            write ("{}return {};".format(indent, np.argmax(tree_.value[node])))
    from sklearn.tree import DecisionTreeClassifier
    
    recurse(0, 1)
    
    write("};")
    write("")


def export_rfc_tojs(rfc:'sklearn.ensemble.RandomForestClassifier',func_template:str):
    dt_list      = []
    estimators   = rfc.estimators_
    n_estimators = len(estimators)
    feat_names   = list(rfc.feature_names_in_)
    
    print(f" - Found {n_estimators} estimators")
    print(f" - Input {len(feat_names)} Features : {feat_names}")
    print("\n-----------\n")
    
    write("",False)
    
    for i,estimator in enumerate(estimators):
        decode_dt_tojs(estimator,feat_names,f"{func_template}{i}")
    
    write(f"let predict_rps = (",end="")
    for i,feature in enumerate(feat_names):
        if i == len(feat_names)-1:
            write(feature,end="")
        else:    
            write(feature,end=",")
    write(") =>{")
    write("    let results = [];")
    write("    let map = { };")
    write(f"    let model = [{','.join([f'{func_template}{i}' for i in range(n_estimators)])}];")
    write("    model.forEach(x => { results.push(x(" + ",".join(feat_names) + "))});")
    write("    results.forEach( x => { if (map[x] != undefined) {map[x] += 1}else{map[x] = 1}});")
    write("    if (map[0] > map[1]){if (map[0] > map[2]){return 0;}else{return 2;}}else {if (map[1] > map[2]){return 1;}else{return 2;}}")
    write("};")
    print("Model saved as :", MODEL_OUT)    

In [25]:
export_rfc_tojs(model,"dt")

 - Found 100 estimators
 - Input 42 Features : ['WRIST_X', 'WRIST_Y', 'THUMB_CMC_X', 'THUMB_CMC_Y', 'THUMB_MCP_X', 'THUMB_MCP_Y', 'THUMB_IP_X', 'THUMB_IP_Y', 'THUMB_TIP_X', 'THUMB_TIP_Y', 'INDEX_FINGER_MCP_X', 'INDEX_FINGER_MCP_Y', 'INDEX_FINGER_PIP_X', 'INDEX_FINGER_PIP_Y', 'INDEX_FINGER_DIP_X', 'INDEX_FINGER_DIP_Y', 'INDEX_FINGER_TIP_X', 'INDEX_FINGER_TIP_Y', 'MIDDLE_FINGER_MCP_X', 'MIDDLE_FINGER_MCP_Y', 'MIDDLE_FINGER_PIP_X', 'MIDDLE_FINGER_PIP_Y', 'MIDDLE_FINGER_DIP_X', 'MIDDLE_FINGER_DIP_Y', 'MIDDLE_FINGER_TIP_X', 'MIDDLE_FINGER_TIP_Y', 'RING_FINGER_MCP_X', 'RING_FINGER_MCP_Y', 'RING_FINGER_PIP_X', 'RING_FINGER_PIP_Y', 'RING_FINGER_DIP_X', 'RING_FINGER_DIP_Y', 'RING_FINGER_TIP_X', 'RING_FINGER_TIP_Y', 'PINKY_MCP_X', 'PINKY_MCP_Y', 'PINKY_PIP_X', 'PINKY_PIP_Y', 'PINKY_DIP_X', 'PINKY_DIP_Y', 'PINKY_TIP_X', 'PINKY_TIP_Y']

-----------

Model saved as : ../models/model.js


### Removing unecessary newlines and indents

In [26]:
text = ""
with open(MODEL_OUT,'r') as file:
    text = "".join(  [ line.strip() for line in file.readlines() ] )

with open(MODEL_OUT,'w') as file:
    file.write(text)